# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

In [ ]:
from pathlib import Path
from db_connections import client,db_neo4j,db_redis
from src import mongo

## Conexión con bases de datos

In [ ]:
# =====================
# TEST DE CONEXIONES
# =====================
print("MongoDB colecciones:", client["clase"].list_collection_names())
print("Neo4j driver activo:", db_neo4j is not None)
print("Redis ping:", db_redis.ping())

## Carga de Datos

In [ ]:
# --- MONGO ---
nombre_base="clase"
nombre_colecciones = ["usuarios","destinos","hoteles","reservas","actividades"]
nombre_archivos= ["usuarios.csv","destinos.csv","hoteles.csv","reservas.csv","actividades.csv"]

for i in range (5):
    ruta=Path("fuentes") / nombre_archivos[i]
    mongo.crear_coleccion(nombre_base,nombre_colecciones[i],recrear=True)
    mongo.cargar_csv_a_coleccion(ruta,nombre_base,nombre_colecciones[i])

##### Contar la cantidad de documentos existentes en cada colección

In [ ]:
for i in range (5):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

##### Listar datos de las colecciones

In [ ]:
# Lista el primer dato de cada colección como prueba de la carga

for i in range (5):
    cursor=mongo.obtener_cursor(nombre_base,nombre_colecciones[i], 1,proyeccion={"_id":0})
    print(f"Colección {nombre_colecciones[i]}:")
    mongo.imprimir_cursor(cursor)